In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#合并数据集
train_df=pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train.csv')
train_cal_df=pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train_calendar.csv')

In [ ]:
train_cal_df['id']=train_cal_df['warehouse']+'_'+train_cal_df['date']

In [ ]:
train_df['date']=pd.to_datetime(train_df['date'])
train_cal_df['date']=pd.to_datetime(train_cal_df['date'])

In [ ]:
train_df['warehouse'].unique()

In [ ]:
train=train_df[train_df['warehouse'].isin(['Prague_1', 'Brno_1', 'Prague_2', 'Prague_3','Budapest_1','Frankfurt_1'])]

In [ ]:
#不要holiday_name
train_cols_moved=['warehouse', 'date','holiday','shops_closed','winter_school_holidays',
       'school_holidays','id','orders']

In [ ]:
train=train[train_cols_moved]

In [ ]:
train.dtypes

In [ ]:
for col in train.select_dtypes(include=['object']).columns:
    train[col]=train[col].astype('category')

In [ ]:
train.head()

In [ ]:
train['month']=train['date'].dt.month.astype('category')
train['weekday']=train['date'].dt.weekday.astype('category')
train['year']=train['date'].dt.year.astype('category')
train['quarter'] = train['date'].dt.quarter.astype('category')
train['is_week51'] = (train['date'].dt.isocalendar().week==51).astype(int).astype('category')
train['is_friday'] = (train['weekday'] == 4).astype(int).astype('category')
train['is_q3'] = (train['quarter'] == 3).astype(int).astype('category')
#train['is_Sunday'] = (train['weekday'] == 6).astype(int).astype('category')


In [ ]:
train.drop(columns=['date'],inplace=True)

In [ ]:
##test

In [ ]:
train.count()

In [ ]:
##test

In [ ]:
#measure
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool

In [ ]:
train.set_index('id',inplace=True)

In [ ]:
#"""
y_train=train['orders']
X_train=train.drop(columns=['orders',])
#"""

In [ ]:
train.select_dtypes(include=['category','object']).columns

In [ ]:
category=['warehouse','month', 'weekday', 'year', 'quarter','is_week51',
       'is_friday', 'is_q3' ]

In [ ]:
#"""
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.2,random_state=44,shuffle=True)
X_valid,X_test,y_valid,y_test=train_test_split(X_valid,y_valid,test_size=0.5,random_state=44,shuffle=True)
#"""

In [ ]:


params = {
    "boosting_type": "gbdt",
    'objective': 'regression',
   'max_depth':7,
'learning_rate':0.03797305659272102,
'subsample_freq':4,
'subsample':0.9996037318275165,
'num_boost_round':2000,
'reg_alpha':1.0007536818669134,
'reg_lambda':9.754591582685322,
'num_leaves':28,
'min_child_samples':15,
'min_child_weight':0.05100440033373066,
    "random_state": 44,
    "extra_trees": True,
    "verbose": -1,
    #后面是dart的参数
    #'drop_rate':0.1,
    #'skip_drop':0.5,
    #'uniform_drop':False,
    #'xgboost_dart_mode':False,
   
}


In [ ]:
#"""
model=lgb.LGBMRegressor(**params)
model.fit(X_train,y_train,
          eval_set=[(X_valid,y_valid)],
          #callbacks=[lgb.log_evaluation(200),lgb.early_stopping(100)],
          categorical_feature=category,
          #eval_metric=mape,
              )
#"""

In [ ]:
"""
for col in category:
    X_train[col] = X_train[col].astype(str).fillna('NaN')
    X_valid[col] = X_valid[col].astype(str).fillna('NaN')
    X_test[col]=X_test[col].astype(str).fillna('NaN')
"""

In [ ]:
#catboost
'''
train_pool = Pool(X_train, y_train,
                      cat_features=category
                     )
val_pool = Pool(X_valid, y_valid,
                    cat_features=category
                   )
    
clf = CatBoostRegressor(learning_rate=0.02, iterations=4000,loss_function='RMSE',
                            )
    
random_seed=3107
    
clf.fit(train_pool, eval_set=val_pool,verbose=100)
'''

In [ ]:
#分层分组抽样
"""
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, StratifiedGroupKFold
y= train["orders"]
years = train["year"]
df_train= train.drop(columns=['id','orders'])
cv = StratifiedGroupKFold(n_splits=3, shuffle=True)

#fitted_models_cat = []
fitted_models_lgb = []

#cv_scores_cat = []
cv_scores_lgb = []

#stability_results_cat = []
stability_results_lgb = []

feature_importance_lgb = []

fold = 0
for idx_train, idx_valid in cv.split(df_train, y,
                                    groups=years
                                    ):
    
    #df_res_cat = pd.DataFrame()
    df_res_lgb = pd.DataFrame()
    
    X_train, y_train = df_train.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid= df_train.iloc[idx_valid], y.iloc[idx_valid]
    """"""
    df_res_cat['WEEK_NUM'] = list(week_valid)
    df_res_cat['target'] = list(y_valid)
    df_res_cat['fold'] = fold
    """"""
    
    #df_res_lgb['WEEK_NUM'] = list(week_valid)
    df_res_lgb['target'] = list(y_valid)
    df_res_lgb['fold'] = fold
    
    fold += 1
    
    """"""
    train_pool = Pool(X_train, y_train,cat_features=cat_cols)
    val_pool = Pool(X_valid, y_valid,cat_features=cat_cols)
    
    clf = CatBoostClassifier(eval_metric='AUC', task_type='GPU', learning_rate=0.03, iterations=n_est)
    
    random_seed=3107
    
    clf.fit(train_pool, eval_set=val_pool,verbose=300)
    fitted_models_cat.append(clf)
    y_pred_valid = clf.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_cat.append(auc_score)
    
    df_res_cat['score'] = list(y_pred_valid)
    
    
    
    X_train[cat_cols] = X_train[cat_cols].astype("category")
    X_valid[cat_cols] = X_valid[cat_cols].astype("category")
    """"""
    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set = [(X_valid, y_valid)],
        categorical_feature=category,
        #callbacks=[wandb_callback()]
        #callbacks=[lgb.log_evaluation(200), lgb.early_stopping(100),] 
    )
    
    fitted_models_lgb.append(model)
    y_pred_valid = model.predict(X_valid)
    #mape指标
    scores = mape(y_valid, y_pred_valid)
    cv_scores_lgb.append(scores)
    #特征重要性
    fold_importance = model.feature_importances_
    feature_importance_lgb.append(fold_importance)
    
    
    df_res_lgb['score'] = list(y_pred_valid)
    
    #stability_results_cat.append(df_res_cat)
    stability_results_lgb.append(df_res_lgb)

#计算特征重要性
avg_feature_importance_lgb = np.mean(feature_importance_lgb, axis=0)
sorted_idx = np.argsort(avg_feature_importance_lgb)[::-1]
print('Feature importance ranking')
for i, idx in enumerate(sorted_idx):
    print(f"{i + 1}. Feature '{df_train.columns[idx]}' - Importance: {avg_feature_importance_lgb[idx]}")

    
    
#print("Catboost CV AUC scores: ", cv_scores_cat)
#print("Maximum Catboost CV AUC score: ", max(cv_scores_cat))


print("Lightgbm CV mape scores: ", cv_scores_lgb)
print("Maximum Lightgbm CV mape score: ", min(cv_scores_lgb),)
"""

In [ ]:
y_pred=pd.Series(model.predict(X_test),index=X_test.index,name='y_Pred')
y_test=pd.Series(y_test,index=X_test.index,name='y_true')
mape(y_test,y_pred)

In [ ]:
test1=pd.concat([X_test,y_pred,y_test],axis=1)
mape_by_warehouse = test1.groupby('warehouse').apply(lambda x: mape(x['y_true'],x['y_Pred']))
print(mape_by_warehouse)

In [ ]:
#慕尼黑部分
def mark_shutdown(row):
    if row['date'].month ==12 and row['date'].day==25:
        return 1
    elif row['date'].weekday()==6:
        return 1
    elif row['holiday_name']in\
    ['Easter Monday', 'Ascension day', 'Corpus Christi','2nd Christmas Day', 'Assumption of the Virgin Mary',
       'New Years Day', 'Christmas Eve', 'Labour Day', 'Whit sunday','Good Friday', 'German Unity Day', '1st Christmas Day', 'Epiphany',
       'Whit monday', 'Peace Festival in Augsburg', 'All Saints Day']:
        return 1
    else:
        return 0

In [ ]:
train_cal_df=train_cal_df[train_cal_df['date'].between('2022-09-25', '2024-03-15')]

In [ ]:
train_mun=pd.merge(train_cal_df,train_df[['date','warehouse','orders',]],on=['date','warehouse'],how='left')

In [ ]:
train_mun=train_mun[train_mun['warehouse']=='Munich_1']

In [ ]:
train_mun['orders']=train_mun['orders'].fillna(0)

In [ ]:
train_mun['shops_closed']=(train_mun['orders']==0).astype(int)

In [ ]:
train_mun['holiday_sunday']=train_mun.apply(mark_shutdown,axis=1)

In [ ]:
train_mun['month']=train_mun['date'].dt.month
train_mun['weekday']=train_mun['date'].dt.weekday
train_mun['year']=train_mun['date'].dt.year-2020
train_mun['quarter'] = train_mun['date'].dt.quarter
train_mun['is_week51'] = (train_mun['date'].dt.isocalendar().week==51).astype(int)
train_mun['is_wednesday'] = (train_mun['weekday'] == 2).astype(int)
train_mun['is_friday'] = (train_mun['weekday'] == 4).astype(int)
train_mun['is_q3'] = (train_mun['quarter'] == 3).astype(int)


In [ ]:
train_mun=train_mun[['date','school_holidays','holiday','holiday_sunday','is_week51','is_wednesday','is_friday',
       'is_q3','shops_closed','weekday','month', 'year', 'quarter','orders']]

In [ ]:
train_mun=train_mun.set_index('date').sort_index()

In [ ]:
#lstm
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,MinMaxScaler,QuantileTransformer,PowerTransformer
from tensorflow.keras import regularizers

In [ ]:
train_norm=train_mun[[ 'school_holidays','holiday','holiday_sunday','is_week51','is_wednesday','is_friday',
       'is_q3','shops_closed','weekday','month', 'year', 'quarter','orders']].values
scaler= PowerTransformer(method='yeo-johnson')
#train_norm=scaler.fit_transform(train_norm)
#只标准化orders
orders_scaled = scaler.fit_transform(train_mun[['weekday','month','year', 'quarter','orders']])
train_norm[:,-5:] = orders_scaled

In [ ]:
def window_dataset(series,window_size,batch_size,shuffle_buffer):
    dataset=tf.data.Dataset.from_tensor_slices(series)
    dataset=dataset.window(window_size+1,shift=1,drop_remainder=True)
    dataset=dataset.flat_map(lambda window:window.batch(window_size +1))
    dataset=dataset.map(lambda window:(window[:-1,:],window[-1,-1]))
    dataset=dataset.shuffle(shuffle_buffer)
    dataset=dataset.batch(batch_size).prefetch(1)
    return dataset
def mape_mun(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    epsilon = tf.keras.backend.epsilon()
    return tf.reduce_mean(tf.abs((y_true - y_pred) / tf.maximum(tf.abs(y_true), epsilon))) * 100

In [ ]:
window_size=7
batch_size=1
shuffle_buffer_size=100

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(44)
np.random.seed(44)
dataset=window_dataset(train_norm,window_size,batch_size,shuffle_buffer_size)

In [ ]:
model_lstm=tf.keras.models.Sequential([
    #tf.keras.layers.Lambda(lambda x:tf.expand_dims(x,axis=-1),input_shape=[None]),
    #tf.keras.layers.LSTM(64,return_sequences=True),
    #tf.keras.layers.LSTM(64),
    #tf.keras.layers.Conv1D(128, kernel_size=30, activation='tanh'),
    tf.keras.layers.Input(shape=[window_size,13]),
    tf.keras.layers.Conv1D(80, kernel_size=1, activation='tanh'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(70,return_sequences=True, 
                                                       #kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0011),
                                                       #kernel_regularizer=regularizers.l2(0.0001)
                                                      )),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,
                                                       #kernel_regularizer=regularizers.l1(0.0001),
                                                       return_sequences=True
                                                      )),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(10,
                                                       #kernel_regularizer=regularizers.l2(0.0001),
                                                       #return_sequences=True
                                                      )),


    tf.keras.layers.Dense(1)
])
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',    
    factor=0.7,            
    patience=3,       
    min_lr=1e-6,         
    verbose=1            
)
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
model_lstm.compile(loss=tf.keras.losses.Huber(),
             optimizer=optimizer,
             metrics=[mape_mun])
history=model_lstm.fit(dataset,epochs=90,callbacks=[reduce_lr],shuffle=True)

In [ ]:
test_df=pd.read_csv("/kaggle/input/rohlik-orders-forecasting-challenge/test.csv")
test_cal_df=pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test_calendar.csv')

In [ ]:
test_cal_df['id']=test_cal_df['warehouse']+'_'+test_cal_df['date']

In [ ]:
test_df['date']=pd.to_datetime(test_df['date'])
test_cal_df['date']=pd.to_datetime(test_cal_df['date'])

In [ ]:
test_cols_moved=['warehouse', 'date','holiday','shops_closed','winter_school_holidays',
       'school_holidays','id']

In [ ]:
#将慕尼黑与其他仓库分开
test=test_df[test_cols_moved]
test=test[test['warehouse'].isin(['Prague_1', 'Brno_1', 'Prague_2', 'Prague_3','Budapest_1','Frankfurt_1'])]
test_mun_or=test_df[test_df['warehouse']=='Munich_1']

In [ ]:
test_mun_cal=test_cal_df[test_cal_df['warehouse']=='Munich_1']

In [ ]:
test_mun=test_mun_cal[test_mun_cal['date'].between(test_mun_or['date'].min(),test_mun_or['date'].max())]

In [ ]:
test_mun['holiday_sunday']=test_mun.apply(mark_shutdown,axis=1)

In [ ]:
test_mun['date']=pd.to_datetime(test_mun['date'])
test_mun['month']=test_mun['date'].dt.month
test_mun['weekday']=test_mun['date'].dt.weekday
test_mun['year']=test_mun['date'].dt.year-2020
test_mun['quarter'] = test_mun['date'].dt.quarter
test_mun['is_week51'] = (test_mun['date'].dt.isocalendar().week==51).astype(int)
test_mun['is_wednesday'] = (test_mun['weekday'] == 2).astype(int)
test_mun['is_friday'] = (test_mun['weekday'] == 4).astype(int)
test_mun['is_q3'] = (test_mun['quarter'] == 3).astype(int)

In [ ]:
#其他仓库
for col in test.select_dtypes(include=['object']).columns:
    test[col]=test[col].astype('category')

In [ ]:
test['date']=pd.to_datetime(test['date'])
test['month']=test['date'].dt.month.astype('category')
test['weekday']=test['date'].dt.weekday.astype('category')
test['year']=test['date'].dt.year.astype('category')
test['quarter'] = test['date'].dt.quarter.astype('category')
test['is_week51'] = (test['date'].dt.isocalendar().week==51).astype(int).astype('category')
test['is_friday'] = (test['weekday'] == 4).astype(int).astype('category')
test['is_q3'] = (test['quarter'] == 3).astype(int).astype('category')
#test['is_Sunday'] = (test['weekday'] == 6).astype(int).astype('category')


In [ ]:
test.drop(columns=['date'],inplace=True)

In [ ]:
"""
for col in category:
    test[col] = test[col].astype(str).fillna('NaN')
"""

In [ ]:
test.head()

In [ ]:
test=test.set_index('id')

In [ ]:
y_test_pred=pd.DataFrame(model.predict(test),index=test.index,columns=['Forecats'])

In [ ]:
#读取模型
"""
y_pred_test = np.zeros(len(test))
for model in fitted_models_lgb:
    y_pred=model.predict(test)
    y_pred_test += y_pred
y_pred_test /= len(fitted_models_lgb)
"""

In [ ]:
sub=pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/solution_example.csv')

In [ ]:
#开始mun预测

In [ ]:
test_mun=test_mun.set_index('date').sort_index()

In [ ]:
test_mun['orders'] = 0

In [ ]:
#标准化
test_mun_nor=test_mun[['school_holidays','holiday','holiday_sunday','is_week51','is_wednesday','is_friday',
       'is_q3','shops_closed','weekday','month', 'year', 'quarter']].values


orders_scaled = scaler.transform(test_mun[['weekday','month','year', 'quarter','orders']])
orders_scaled=orders_scaled[:,:4]
test_mun_nor[:,-4:] = orders_scaled

In [ ]:
test_mun_nor[:5]

In [ ]:
len(test_mun_or)

In [ ]:
#递归预测-多特征
forecast=[]
input_data=train_norm[-window_size:]
for time in range(len(test_mun_nor)):
    features=test_mun_nor[time,0:12]
    input_data_exp= input_data.reshape(1,window_size,13)
    prediction=model_lstm.predict(input_data_exp,verbose=0)
    forecast.append(prediction[0,0])
    feature_update = np.concatenate((input_data[1:,0:12],np.expand_dims(features, axis=0)), axis=0)
    orders_update = np.concatenate((input_data[1:,[12]],prediction.reshape(1, -1)), axis=0)
    input_data=np.concatenate((feature_update,orders_update),axis=1)

forecast=np.array(forecast)

In [ ]:
forecast_tr=np.concatenate((np.zeros((len(test_mun),4)),forecast.reshape(-1,1)), axis=1)

In [ ]:
forecast_tr=scaler.inverse_transform(forecast_tr.reshape(-1,5))


In [ ]:
test_mun_pred=pd.DataFrame(forecast_tr[:,-1],test_mun.index,columns=['orders'])

In [ ]:
test_mun_or=test_mun_or.set_index('id')

In [ ]:
test_mun_pred['id']='Munich_1_'+test_mun_pred.index.strftime('%Y-%m-%d')

In [ ]:
test_mun_pred=test_mun_pred.set_index('id')

In [ ]:
test_mun_pred[:5]

In [ ]:
#开始匹配
condition1=sub['id'].str.contains('Prague_1|Brno_1|Prague_2|Prague_3|Budapest_1|Frankfurt_1')

In [ ]:
sub.loc[condition1,'orders']=sub.loc[condition1].merge(y_test_pred,left_on=['id'],right_index=True,how='left')['Forecats'].astype('float64')

In [ ]:
condition2=sub['id'].str.contains('Munich_1')

In [ ]:
sub.loc[condition2,'orders']=sub.loc[condition2].merge(test_mun_pred,left_on=['id'],right_index=True,how='left')['orders_y'].astype('float64')

In [ ]:
sub.to_csv('submission.csv',index=False,float_format='%.8f')

In [ ]:
test.dtypes

In [ ]:
train.dtypes

In [ ]:
sub[sub['id'].str.contains('Mun')]

In [ ]:
sub